# Homework 4
---

Released on Nov 26 -- due by *Dec 3, 11:59pm*.

In this homework assignment we will develop a copy number caller based on a Hidden Markov Model. In addition, we will implement a hierarchical clustering algorithm. The data that we consider is from a previously published [breast tumor](http://dx.doi.org/10.1038/nature09807), from which 100 single cells have been sequenced. In this homework assignment we restrict our attention to a subset of 25 cells.

_Instructions:_ There are four questions, worth a total of 100 points. In addition, there are two bonus questions (Q2e and Q4c), each worth 5 points. Insert your answer after each question in the designated box. Replace comments with code.

Hand in your work by emailing the Jupyter notebook (please rename as "LASTNAME_FIRSTNAME.ipynb") to Anusri and myself.

---

## Question 1: Data preparation [20 points]

We start by parsing the data from `data.tsv` using pandas.

In [ ]:
%matplotlib inline
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from __future__ import print_function

In [ ]:
df = pd.read_csv("data.tsv", sep="\t", index_col=0)
df.head()

This dataset consists of the following 25 cells:

In [ ]:
cells = list(df.columns[3:])
sys.stdout.write("Cells:")
for idx, cell in enumerate(cells):
    if idx % 5 == 0:
        sys.stdout.write("\n")
    sys.stdout.write(cell + " ")

The reads of each cell are put in bins. As seen in the command below, there are 5363 bins.

In [ ]:
len(df.index)

We plot a histogram of the bin sizes as follows, from which we can see that most of the bins have length 500,000.

In [ ]:
(df['END'] - df['START'] + 1).hist(bins=1000, xrot=90)
plt.xlim((400000, 1000000))

### _Q1a:_ What is the minimum, median and maximum bin size? [4 points]

In [ ]:
# TODO: insert code to print the min, median and max bin size

_Answer:_ 
* min: XXX
* median: XXX
* max: XXX

We retain bins with lengths between 450,000 and 500,000 using the following command.

In [ ]:
df = df[(450000 <= (df['END'] - df['START'] + 1)) & ((df['END'] - df['START'] + 1) <= 550000)]

### _Q1b:_ The column `CHR` contains the chromosome label. Do these cells originate from a male or female patient? [4 points]

In [ ]:
# TODO: write code here to print unique list of values in 'CHR' column

_Answer:_ male/female

Next, we remove the sex chromosomes such that each bin in a normal cell has copy number 2.
### _Q1c:_ How many bins occur on autosomal chromosomes? [4 points]

In [ ]:
# TODO: insert code to generate a list of chromosome labels corresponding to autosomes, e.g. ['chr1', 'chr2', ...]
autosomes = None

In [ ]:
len(df[df['CHR'].isin(autosomes)])

_Answer:_ XXX bins

In [ ]:
# we retain only autosomes
df = df[df['CHR'].isin(autosomes)]

Due to variablity in library preparation, there are differences in the total number of reads per cell. We construct a new dataframe with the total number of reads, and plot a histogram as follows.

In [ ]:
df_total_reads = pd.DataFrame.from_records([(cell, int(df[cell].sum())) for cell in cells], columns=['cell', 'total'])
df_total_reads.hist(column="total", bins=15, xrot=90)

### _Q1d:_ Based on the above plot, we decide to exclude the six cells with a total read count of less than 3,000,000 reads. Which six cells are excluded? [4 points]

In [ ]:
cells_to_keep = list(df_total_reads[df_total_reads['total'] >= 3000000]['cell'])

In [ ]:
# TODO: insert code to filter down to cells with less 3,000,000 reads
df_removed_cells = None

In [ ]:
df_removed_cells

_Answer:_ 
1. XXX with YYY reads; 
2. XXX with YYY reads; 
3. XXX with YYY reads; 
4. XXX with YYY reads; 
5. XXX with YYY reads;
6. XXX with YYY reads.

In [ ]:
# we retain only cells with >= 300,000 reads
df = df[list(df.columns)[:3] + cells_to_keep]

The key concept is that the majority of bins in a cell are diploid. Since the bins have the same length and under the assumption of uniform genome coverage, we expect the most frequent read count, or the mode, to correspond to the diploid state. We identify the mode by discretizing the read counts for each cell into 20 bins. We then divide the read count for each bin in a cell by the mode.

### _Q1e:_ Complete the below code fragment to identify the mode of each cell and to subsequently normalize the read counts. [4 points]

In [ ]:
nr_bins = 20
df2 = df.copy(deep=True)
for cell in cells_to_keep:
    # insert code to discretize read count (e.g. first integer division by bin_size, then multiply by bin_size)
    largest = None
    bin_size = None
    df2[cell] = None
    
df_mode = pd.DataFrame.from_records([(cell, int(df2[cell].mode())) for cell in cells_to_keep], 
                                    columns=['cell', 'mode'])

In [ ]:
# TODO: insert code to divide each read count by the mode of the corresponding cell
for cell in cells_to_keep:
    df[cell] = None

In [ ]:
# The following commented out code generates data_filtered.tsv, which is part of the zip file of this assignment. 
# If your code is correct the resulting file should be identical to the provided file.
# df.to_csv("data_filtered_normalized.tsv", sep="\t")

## Question 2: Viterbi algorithm [50 points]

In this question we use the Viterbi algorithm to compute a copy number for each bin. The set $Q$ of hidden states corresponds to integer copy numbers. We consider $Q=\{0,\ldots,10\}$. The set $\Sigma$ of emitted symbols are normalized read counts. Since this an infinite set, we use a probability distribution function for each integer copy number to define the emission probabilities. Specifically, for copy number $c \in \{0,\ldots,10\}$ we use a normal distribution with standard deviation $\sigma = 0.1$ centered around mean $\mu_c = c / 2$.

In [ ]:
df_norm = pd.read_table("data_filtered_normalized.tsv", sep="\t", index_col=0)

In [ ]:
df_norm.head()

In [ ]:
def gauss(x, mu, sigma, A):
    return A*np.exp(-(x-mu)**2/2/sigma**2)

values = [val for sublist in [list(df_norm[cell]) for cell in list(df_norm.columns[3:])] for val in sublist]
count, bins, ignored = plt.hist(values, 100, density=True)

sigma = 0.1

plt.plot(np.array([0,.05,.1,.15,.2]),gauss(np.array([0,.05,.1,.15,.2]), 0, sigma, .01),color='black',lw=3,label='copy number 0')
plt.plot(bins,gauss(bins, 0.5, sigma, .25),color='red',lw=3,label='copy number 1')
plt.plot(bins,gauss(bins, 1, sigma, 1.3),color='blue',lw=3,label='copy number 2')
plt.plot(bins,gauss(bins, 1.5, sigma, .7),color='cyan',lw=3,label='copy number 3')
plt.plot(bins,gauss(bins, 2, sigma, .25),color='orange',lw=3,label='copy number 4')
plt.plot(bins,gauss(bins, 2.5, sigma, .1),color='purple',lw=3,label='copy number 5')
plt.plot(bins,gauss(bins, 3, sigma, .05),color='pink',lw=3,label='copy number 6')
plt.plot(bins,gauss(bins, 3.5, sigma, .05),color='gray',lw=3,label='copy number 7')
plt.plot(bins,gauss(bins, 4, sigma, .05),color='yellow',lw=3,label='copy number 8')
plt.plot(bins,gauss(bins, 4.5, sigma, .05),color='pink',lw=3,label='copy number 9')
plt.plot(bins,gauss(bins, 5, sigma, .05),color='black',lw=3,label='copy number 10')

plt.legend()
plt.gcf().set_size_inches(10, 6)

In [ ]:
from scipy.stats import norm
max_copy_number = 10

def emissionLogProb(copy_number, norm_count):
    sigma = 0.1
    mu = copy_number / 2.

    # Compute Pr(|X_c - norm_count| <= 0.01 | c)  
    low = norm.cdf(norm_count - 0.01, mu, sigma)
    up = norm.cdf(norm_count + 0.01, mu, sigma)
    
    # prevent probability of 0
    prob = max(up - low, 0.0001)
    
    return np.log(prob)

def transitionLogProb(current_copy_number, next_copy_number):
    stay_prob = 0.99999
    if current_copy_number == next_copy_number:
        return np.log(stay_prob)
    elif 0 <= next_copy_number <= max_copy_number:
        return np.log((1 - stay_prob) / max_copy_number)
    else:
        return np.log(0)

def initialLogProb(copy_number):
    if 0 <= copy_number <= max_copy_number:
        return np.log(1./11)
    else:
        return np.log(0)

### _Q2a:_ Compute the joint log probability of each cell having copy number 2 in all bins. [10 points]

_Hint:_ Make use of the functions `emissionLogProb`, `initialLogProb` and `transitionLogProb`.

In [ ]:
for cell in list(df_norm.columns[3:]):
    log_prob = 0
    for idx, norm_count in df_norm[cell].iteritems():
        # TODO: insert code here
        # Hint: idx == 0 is the initial item.
        pass
    
    print(cell, log_prob)

### _Q2b:_ Determine the maximum joint log probability of each cell using the Viterbi algorithm. [20 points]

_Hint:_ Complete the function `viterbi` and `max_joint_prob`, making use of the functions `emissionLogProb`, `initialLogProb` and `transitionLogProb`.

In [ ]:
def viterbi(df_norm, cell):
    # Positions
    bins = sorted(list(df_norm.index))

    # Set of states (copy numbers)
    Q = range(max_copy_number+1)
    
    # Initialization v[copy_number][bin] = 0
    v = [ { bin : 0 for bin in bins } for c in range(max_copy_number + 1) ]
    
    for idx, bin in enumerate(bins):
        norm_count = float(df.loc[bin][cell])
        
        # TODO: insert code here
        pass

    return v

def max_joint_prob(df_norm, v):
    # Positions
    bins = sorted(list(df_norm.index))
    
    # Set of states (copy numbers)
    Q = range(max_copy_number+1)
    last_bin = bins[-1]
    
    # TODO: insert code here
    return None

V = {}
for cell in list(df_norm.columns[3:]):
    V[cell] = viterbi(df_norm, cell)
    print(cell, "--", "max prob:", max_joint_prob(df_norm, V[cell]))

### _Q2c:_ Compute the most likely copy number profile for each cell. [10 points]

_Hint:_ Hint perform a backtrace or extend the above `viterbi` function to maintain backpointers.

In [ ]:
def viterbi_bt(df_norm, cell):
    # Positions
    bins = sorted(list(df_norm.index))

    # Set of states (copy numbers)
    Q = range(max_copy_number+1)
    
    # Initialization v[copy_number][bin] = 0
    v = [ { bin : 0 for bin in bins } for c in range(max_copy_number + 1) ]
    bt = [ { bin : None for bin in bins } for c in range(max_copy_number + 1) ]
    
    for idx, bin in enumerate(bins):
        # TODO: insert code here
        pass

    return v, bt

bins = sorted(list(df_norm.index))
Q = range(max_copy_number+1)

V = {}
BT = {}
C = {}
for cell in list(df_norm.columns[3:]):
    V[cell], BT[cell] = viterbi_bt(df_norm, cell)
    C[cell] = {}
    
    max_prob = max_joint_prob(df_norm, V[cell])
    final_c = None
    final_bin = bins[-1]
    for c in Q:
        if V[cell][c][final_bin] == max_prob:
            final_c = c
    
    C[cell][final_bin] = final_c
    
    for idx in range(len(bins)-2, -1, -1):
        bin = bins[idx]
        next_bin = bins[idx + 1]
        next_bin_c = C[cell][next_bin]
        C[cell][bin] = BT[cell][next_bin_c][next_bin]
    
    print(cell, "--", "max prob:", max_joint_prob(df_norm, V[cell]))

### _Q2d:_ Which cells are normal cells (i.e. have copy number 2 for each bin)? [10 points]

_Hint:_ Use `visualizeCopyNumbers(C, bins)`.

_Answer:_ XXX

In [ ]:
bins = sorted(list(df_norm.index))

In [ ]:
def visualizeCopyNumbers(C, bins):
    nrRows = 5
    nrCols = 4
    for idx, cell in enumerate(C):
        ax = plt.subplot(nrRows, nrCols, idx+1)
        plt.step(range(len(bins)), C[cell].values())
        ax.set_title(cell)
        ax.set_ylim((0, 10))
        ax.set_xlabel("bin")
        ax.set_ylabel("copy number")
    plt.gcf().set_size_inches(30, 20)
    plt.tight_layout()
    plt.savefig("viterbi.pdf")

In [ ]:
visualizeCopyNumbers(C, bins)

### _Q2e -- Bonus:_ Extend the plotting function to delineate chromosomes using vertical lines. Do not forget to update x-axis labels to show chromosomes. [5 points]

## Question 3: Forward algorithm [10 points]

### _Q3a:_ Compute the marginal log probability of the observations using the forward algorithm. [10 points]
_Hint:_ Be aware that recurrence of the forward algorithm includes a summation, but we are operating in log space.

In [ ]:
def forward(df_norm, cell):
    # Positions
    bins = sorted(list(df_norm.index))

    # Set of states (copy numbers)
    Q = range(max_copy_number+1)
    
    # Initialization f[copy_number][bin] = 0
    f = [ { bin : 0 for bin in bins } for c in range(max_copy_number + 1) ]
    
    for idx, bin in enumerate(bins):
        norm_count = float(df.loc[bin][cell])
        
        # TODO: insert code here
        pass

    return f

def marginal_log_prob(df_norm, f):
    # Positions
    bins = sorted(list(df_norm.index))
    
    # Set of states (copy numbers)
    Q = range(max_copy_number+1)
    last_bin = bins[-1]
    
    # insert code here
    return None

F = {}
for cell in list(df_norm.columns[3:]):
    F[cell] = forward(df_norm, cell)
    print(cell, "--", "marginal log prob:", marginal_log_prob(df_norm, F[cell]))

## Question 4: Hierarchical clustering [20 points]
The Python package `scipy` implements hierarchical clustering. We will use this method to perform single linkage cluster with the Manhattan (also known as cityblock) distance metric.

In [ ]:
C_values = [C[cell].values() for cell in C]
C_labels = C.keys()

In [ ]:
from scipy.cluster import hierarchy
Z = hierarchy.linkage(C_values, 'single', 'cityblock')
plt.figure()
dn = hierarchy.dendrogram(Z, labels=C_labels, leaf_rotation=90)

The goal of this question is to reimplement this algorithm. We start by computing the initial distance matrix.

### _Q4a:_ Compute the initial distance matrix using Manhattan distance. [10 points]

In [ ]:
def manhattan(C, cell_1, cell_2, bins):
    # TODO: insert code here
    return None

bins = sorted(list(df_norm.index))
dist = {}
for cell_1 in C.keys():
    dist[cell_1] = {}
    for cell_2 in C.keys():
        dist[cell_1][cell_2] = manhattan(C, cell_1, cell_2, bins)

### _Q4b:_ Implement single linkage clustering. [10 points]
_Hint:_ The documentation for the required output format is described here: https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html#scipy.cluster.hierarchy.linkage

In [ ]:
import copy
def cluster(distances):
    distances = copy.deepcopy(distances)
    clusters = set(distances.keys())
    n = len(clusters)
    cluster2idx = { cell : idx for idx, cell in enumerate(clusters) }
    idx2cluster = distances.keys()
    Z = np.empty((0, 4), float)
    membership = [ set([cluster2idx[cell]]) for cell in clusters ]
    while len(clusters) > 1:
        # TODO: insert code to identify pair (c1, c2) with minimum distance dist
        dist, c1, c2 = None, None, None

        idx_c1 = cluster2idx[c1]
        idx_c2 = cluster2idx[c2]
        new_cluster = n + len(Z)
        new_cluster_idx = len(idx2cluster)

        membership.append( membership[idx_c1] | membership[idx_c2])
        Z = np.append(Z, 
                      np.array([[cluster2idx[c1], cluster2idx[c2], dist, len(membership[new_cluster_idx])]]), 
                      axis=0)
        
        # TODO: insert code to update distances
        distances[new_cluster] = {}


        clusters.add(new_cluster)
        cluster2idx[new_cluster] = len(idx2cluster)
        idx2cluster.append(new_cluster)

        clusters.remove(c1)
        clusters.remove(c2)
        
    return Z

In [ ]:
Z = cluster(dist)
plt.figure()
dn = hierarchy.dendrogram(Z, labels=C_labels, leaf_rotation=90)

### _Q4c -- Bonus:_ Implement complete linkage clustering. [5 points]
_Hint:_ The documentation for the required output format is described here: https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html#scipy.cluster.hierarchy.linkage. Your plot should be identical to the below scipy plot.

In [ ]:
from scipy.cluster import hierarchy
Z = hierarchy.linkage(C_values, 'complete', 'cityblock')
plt.figure()
dn = hierarchy.dendrogram(Z, labels=C_labels, leaf_rotation=90)